## Cleaning

In [73]:
%%capture

%run './lib/init.ipynb'
from lib import utilities as util
from pyspark.sql import functions as F
import pandas as pd

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

sc = SparkContext.getOrCreate()
spark = SparkSession( sc )

In [74]:
filterDf  = util.load_df('filterDf')

studyVars = [
    'item_type_code_name',
    'number_of_distinct_instance_items',
    'item_attempt_duration_mins',
    'student_duration_mins',
    'timeliness_duration_mins'
]
cleaningVars = [
    'raw_score',
    'item_attempt_end_datetime_utc',
    'item_attempt_start_datetime_utc',
    'assessment_item_response_id'
]

###### Create dataset of only study variables

In [75]:
cleanDf = filterDf.select(*studyVars, *cleaningVars)
beforeCnt = cleanDf.count()
beforeCnt

64838

###### Cleaning Tasks

In [76]:
util.list_todo(False)

{'todo': 'Remove null values in response_correctness', 'finished': False}


##### Before Training Validation Split

In [77]:
stepsDf = pd.DataFrame(columns = ['description', 'removed', 'imputed', 'data', 'function'])

###### Remove [unassigned] in response_correctness

In [78]:
# Clean
cleanDf = util.remove_unassigned_response_correctness(cleanDf)

afterCnt = cleanDf.count()
afterCnt

63272

In [79]:
# Update status
def update_status(df, description, function, removed, imputed, data = 'all'):
    util.finish_todo(description)
    return df.append({'description' : description, 'removed' : removed, 'imputed' : imputed, 'function' : function, 'data': data }, ignore_index = True)


stepsDf = update_status(
    stepsDf,
    "Remove [unassigned] in response_correctness",
    'remove_unassigned_response_correctness',
    beforeCnt - afterCnt,
    0
)
beforeCnt = afterCnt

Finished: Remove [unassigned] in response_correctness


###### Remove student start or stop null dates

In [80]:
# Clean
cleanDf = util.remove_null_student_dates(cleanDf)

afterCnt = cleanDf.count()
afterCnt

62835

In [81]:
# Update status

stepsDf = update_status(
    stepsDf,
    "Remove student start or stop null dates",
    'remove_null_student_dates',
    beforeCnt - afterCnt,
    0
)
beforeCnt = afterCnt

Finished: Remove student start or stop null dates


###### Remove attempt stop dates before start date

In [82]:
# Clean
cleanDf = util.remove_attempt_stop_dates_before_start_dates(cleanDf)

afterCnt = cleanDf.count()
afterCnt

62827

In [83]:
# Update status

stepsDf = update_status(
    stepsDf,
    "Remove attempt stop dates before start date",
    'remove_attempt_stop_dates_before_start_dates',
    beforeCnt - afterCnt,
    0
)
beforeCnt = afterCnt

Finished: Remove attempt stop dates before start date


###### Create target variable from raw_score

In [84]:
cleanDf = util.create_target_var_from_raw_score(cleanDf)

##### Create training and validation datasets

In [85]:
trainDf, validDf = cleanDf.select('target', *studyVars).randomSplit([0.75, 0.25], 11112020)

##### Training count

In [86]:
trainDf.count()

47221

##### Validation count

In [87]:
validDf.count()

15606

###### Imputation function

In [88]:
def post_split_imputation(daDf, stDf, data):
    daDf = util.add_swoe(daDf, 'target', 'item_type_code_name', 24)
    stDf = update_status(
        stDf,
        "Reduce number of levels in item_type_code_name",
        'add_swoe',
        0,
        0,
        data
    )

    daDf = util.impute_student_duration(daDf)
    stDf = update_status(
        stDf,
        "Impute max durations outliers in student attempt times",
        'impute_student_duration',
        0,
        daDf.filter( F.col('student_duration_mins_imputed') ).count(),
        data
    )

    daDf = util.impute_timeliness_duration(daDf)
    stDf = update_status(
        stDf,
        "Impute timeliness durations min and max outliers",
        'impute_timeliness_duration',
        0,
        daDf.filter( F.col('timeliness_duration_mins_imputed') ).count(),
        data
    )

    daDf = util.impute_item_attempt_duration(daDf)
    stDf = update_status(
        stDf,
        "Impute max durations outliers in item attempt times",
        'impute_item_attempt_duration',
        0,
        daDf.filter( F.col('item_attempt_duration_mins_imputed') ).count(),
        data
    )


    return daDf, stDf

###### Impute Training Data

In [89]:
trainDf, stepsDf = post_split_imputation(trainDf, stepsDf, 'train')

Finished: Reduce number of levels in item_type_code_name
Finished: Impute max durations outliers in student attempt times
Finished: Impute timeliness durations min and max outliers
Finished: Impute max durations outliers in item attempt times


###### Impute Validation Data

In [90]:
validDf, stepsDf = post_split_imputation(validDf, stepsDf, 'valid')

Finished: Reduce number of levels in item_type_code_name
Finished: Impute max durations outliers in student attempt times
Finished: Impute timeliness durations min and max outliers
Finished: Impute max durations outliers in item attempt times


###### Imputation Counts

In [91]:
print ("Training:")
total = trainDf.count()
print ("  total:", total)
cnt = trainDf.filter( F.col('student_duration_mins_imputed')
                | F.col('timeliness_duration_mins_imputed')
                | F.col('item_attempt_duration_mins_imputed')
                ).count()
print ("  observations:", cnt, round(cnt/total*100), '%')

cnt = trainDf.filter( F.col('student_duration_mins_imputed')
                & F.col('timeliness_duration_mins_imputed')
                & F.col('item_attempt_duration_mins_imputed')
                ).count()
print ("  common:", cnt, round(cnt/total*100), '%')

print ("Validation:")
total = validDf.count()
print ("  total:", total)
cnt = validDf.filter( F.col('student_duration_mins_imputed')
                | F.col('timeliness_duration_mins_imputed')
                | F.col('item_attempt_duration_mins_imputed')
                ).count()
print ("  observations:", cnt, round(cnt/total*100), '%')

cnt = validDf.filter( F.col('student_duration_mins_imputed')
                & F.col('timeliness_duration_mins_imputed')
                & F.col('item_attempt_duration_mins_imputed')
                ).count()
print ("  common:", cnt, round(cnt/total*100), '%')

Training:
  total: 47221
  observations: 15608 33 %
  common: 249 1 %
Validation:
  total: 15606
  observations: 5318 34 %
  common: 59 0 %


###### Cleaning Steps

In [92]:
stepsDf

,description,removed,imputed,data,function
0,Remove [unassigned] in response_correctness,1566,0,all,remove_unassigned_response_correctness
1,Remove student start or stop null dates,437,0,all,remove_null_student_dates
2,Remove attempt stop dates before start date,8,0,all,remove_attempt_stop_dates_before_start_dates
3,Reduce number of levels in item_type_code_name,0,0,train,add_swoe
4,Impute max durations outliers in student attem...,0,9998,train,impute_student_duration
5,Impute timeliness durations min and max outliers,0,3791,train,impute_timeliness_duration
6,Impute max durations outliers in item attempt ...,0,6708,train,impute_item_attempt_duration
7,Reduce number of levels in item_type_code_name,0,0,valid,add_swoe
8,Impute max durations outliers in student attem...,0,3300,valid,impute_student_duration
9,Impute timeliness durations min and max outliers,0,1334,valid,impute_timeliness_duration


In [93]:
util.save_df(validDf, 'validDf')
util.save_df(trainDf, 'trainDf')
util.save_df(spark.createDataFrame(stepsDf), 'stepsDf')